In [1]:
# importing libraries
from sys import exit
from heapq import heapify,heappush,heappop
import pygame
import time
import random

pygame 2.1.2 (SDL 2.0.18, Python 3.10.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

# define Action space
class Actions:
    UP = 1
    DOWN = 2
    LEFT = 3
    RIGHT = 4
    actions=[UP,DOWN,LEFT,RIGHT]


    # Check is it a correct action
def is_valid_action(snake_body,window_x,window_y):
    if snake_body[0][0] < 0 or snake_body[0][0] > window_x-10:
        return False
    if snake_body[0][1] < 0 or snake_body[0][1] > window_y-10:
        return False
    for block in snake_body[1:]:
        if snake_body[0][0] == block[0] and snake_body[0][1] == block[1]:
            return False
    #else 
    return True

# define search Node(inefficient way)
class Node:
    def __init__(self,parent,snake_body,level,action):
        self.parent=parent
        self.snake_body=snake_body
        self.level=level
        self.action=action
        self.cost=0
        self.children=[]
    
    #define heuristic function
    def heuristic(self,fruit_position):
        return abs(self.snake_body[0][0]-fruit_position[0])+abs(self.snake_body[0][1]-fruit_position[1])
    
    # claculate cost and store it in the node
    def calcCost(self,fruit_position):
        self.cost= self.heuristic(fruit_position)
    # explore all available action
    def generate_children(self,window_x,window_y):
        for action in Actions.actions:
            # Moving the snake
            P_body=self.snake_body
            snake_body=[[P_body[i-1][0],P_body[i-1][1]] if i!=0 else [P_body[0][0],P_body[0][1]] for i in range(len(P_body))]
            if action == Actions.UP:
                snake_body[0][1] -= 10
            elif action == Actions.DOWN:
                snake_body[0][1] += 10
            elif action == Actions.LEFT:
                snake_body[0][0] -= 10
            elif action == Actions.RIGHT:
                snake_body[0][0] += 10
            if(is_valid_action(snake_body,window_x,window_y)):
                self.children.append(Node(self,snake_body,self.level+1,action))
        
        return self.children
    # Check is the snake postion equal the other snake position
    def __eq__(self, other):# we didn't use it for performace
        for i in range(len(self.snake_body)):
            for j in range(2):
                if self.snake_body[i][j]!=other.snake_body[i][j]:
                    return False
        return True
    # To meet heapfiy requirement
    def __lt__ (self,other):
        return self.cost<other.cost
       
          
        


# return the solution
def solution_path(node):
    path=[node.action]
    while node.parent is not None:
        node=node.parent
        path.append(node.action)
    return path[::-1]

# 
def is_in_queue(node,queue):
    for state in queue:
        if node==state[1]: # since it's a tuple (cost, node)
            return True
    return False





            

In [3]:

def A_star_search(snake_body,action,fruit_position,window_x,window_y):
    root=Node(parent=None,snake_body=snake_body,level=0,action=action)
    root.calcCost(fruit_position)
    queue=[(root.cost, root)] # the cost is the Priority (cost, refrance to state)
    heapify(queue) 

    while len(queue) !=0: # while queue is not empty
        currunt=heappop(queue) 
        if currunt[1].heuristic(fruit_position)==0: # check goal
            children=currunt[1].generate_children(window_x,window_y)
            if len(children)!=0:# check children exsist
                for child in children:
                    children=child.generate_children(window_x,window_y)
                    if len(children)!=0:
                        return solution_path(currunt[1])
            
        
        for child in currunt[1].generate_children(window_x,window_y):
                child.calcCost(fruit_position)
                #if not is_in_queue(child,queue) and child.cost<=currunt[0]:
                heappush(queue,(child.cost, child))

    return None

Game is copied from : https://www.geeksforgeeks.org/snake-game-in-python-using-pygame-module/

In [4]:

snake_speed = 70

# Window size
window_x = 300
window_y = 200

# defining colors
black = pygame.Color(0, 0, 0)
white = pygame.Color(255, 255, 255)
red = pygame.Color(255, 0, 0)
green = pygame.Color(0, 255, 0)
blue = pygame.Color(0, 0, 255)
# Initialising pygame
pygame.init()

# Initialise game window
pygame.display.set_caption('GeeksforGeeks Snakes')
game_window = pygame.display.set_mode((window_x, window_y))

# FPS (frames per second) controller
fps = pygame.time.Clock()

# defining snake default position
snake_position = [100, 50]

# defining first 4 blocks of snake body
snake_body = [[100, 50],
			[90, 50],
			[80, 50],
			[70, 50]
			]
# fruit position
fruit_position = [random.randrange(1, (window_x//10)) * 10,
				random.randrange(1, (window_y//10)) * 10]

fruit_spawn = True
# setting default snake direction towards
# right
direction = Actions.RIGHT
change_to = direction

# initial score
score = 0

# displaying Score function
def show_score(choice, color, font, size):

	# creating font object score_font
	score_font = pygame.font.SysFont(font, size)
	
	# create the display surface object
	# score_surface
	score_surface = score_font.render('Score : ' + str(score), True, color)
	
	# create a rectangular object for the text
	# surface object
	score_rect = score_surface.get_rect()
	
	# displaying text
	game_window.blit(score_surface, score_rect)

# game over function
def game_over():

	# creating font object my_font
	my_font = pygame.font.SysFont('times new roman', 20)
	
	# creating a text surface on which text
	# will be drawn
	game_over_surface = my_font.render(
		'Your Score is : ' + str(score), True, red)
	
	# create a rectangular object for the text
	# surface object
	game_over_rect = game_over_surface.get_rect()
	
	# setting position of the text
	game_over_rect.midtop = (window_x/2, window_y/4)
	
	# blit will draw the text on screen
	game_window.blit(game_over_surface, game_over_rect)
	pygame.display.flip()
	
	# after 2 seconds we will quit the program
	time.sleep(2)
	
	# deactivating pygame library
	pygame.quit()
	
	# quit the program
	exit()


# Main Function
while True:
#  handling key events
# 	for event in pygame.event.get():
# 		if event.type == pygame.KEYDOWN:
# 			if event.key == pygame.K_UP:
# 				change_to = Actions.UP
# 			if event.key == pygame.K_DOWN:
# 				change_to = Actions.DOWN
# 			if event.key == pygame.K_LEFT:
# 				change_to = Actions.LEFT
# 			if event.key == pygame.K_RIGHT:
# 				change_to = Actions.RIGHT
    pygame.event.clear()
    path=A_star_search(snake_body,direction,fruit_position,window_x,window_y)
    
    if path is None:
        game_over()
        
    try:
        change_to = path[1]
    except:
        print(path[0])
        change_to = path[0]
	# If two keys pressed simultaneously
	# we don't want snake to move into two
	# directions simultaneously
    if change_to == Actions.UP and direction != Actions.DOWN:
        direction = Actions.UP
    if change_to == Actions.DOWN and direction != Actions.UP:
        direction = Actions.DOWN
    if change_to == Actions.LEFT and direction != Actions.RIGHT:
        direction = Actions.LEFT
    if change_to == Actions.RIGHT and direction != Actions.LEFT:
        direction = Actions.RIGHT


    


	# Moving the snake
    if direction == Actions.UP :
        snake_position[1] -= 10
    if direction == Actions.DOWN :
        snake_position[1] += 10
    if direction == Actions.LEFT :
        snake_position[0] -= 10
    if direction == Actions.RIGHT :
        snake_position[0] += 10

	# Snake body growing mechanism
	# if fruits and snakes collide then scores
	# will be incremented by 10
    snake_body.insert(0, list(snake_position))
    if snake_position[0] == fruit_position[0] and snake_position[1] == fruit_position[1]:
        score += 10
        fruit_spawn = False
    else:
        snake_body.pop()

    if not fruit_spawn:
        path=None
        fruit_position = [random.randrange(1, (window_x//10)) * 10,
                            random.randrange(1, (window_y//10)) * 10]

    fruit_spawn = True
    game_window.fill(black)
    pygame.draw.rect(game_window, red,
                pygame.Rect(snake_body[0][0], snake_body[0][1], 10, 10))
    for pos in snake_body[1:]:
        pygame.draw.rect(game_window, green,
                pygame.Rect(pos[0], pos[1], 10, 10))
    pygame.draw.rect(game_window, white, pygame.Rect(
            fruit_position[0], fruit_position[1], 10, 10))

	# Game Over conditions
    if snake_position[0] < 0 or snake_position[0] > window_x-10:
        game_over()
    if snake_position[1] < 0 or snake_position[1] > window_y-10:
        game_over()

	# Touching the snake body
    for block in snake_body[1:]:
        if snake_position[0] == block[0] and snake_position[1] == block[1]:
            game_over()

	# displaying score countinuously
    show_score(1, white, 'times new roman', 20)

	# Refresh game screen
    pygame.display.update()

	# Frame Per Second /Refresh Rate
    fps.tick(snake_speed)


SystemExit: 

C:\Users\nayef\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3450: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
